# Introduction to Structured Outputs

https://cookbook.openai.com/examples/structured_outputs_intro


In [1]:
%pip install openai dotenv smolagents -U -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

import dotenv
from openai import OpenAI

dotenv_path = os.path.join('.env')
dotenv.load_dotenv(dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")
MODEL = 'gpt-4o-mini'

client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

## Example 1: Math Tutor


In [5]:
from textwrap import dedent

math_tutor_prompt = """You are tutor and an expert in solving mathematical problems.
You will be given a math problem. Solve the problem step-by-step with reasoning. For each step provide an explanation and output"""


def get_math_solution(question: str):

    response_format = {"type": "json_schema",
                       "json_schema": {
                           "name": "math_reasoning",
                           "schema": {
                               "type": "object",
                               "properties": {
                                   "steps": {
                                       "type": "array",
                                       "items": {
                                           "type": "object",
                                           "properties": {
                                               "explanation": {
                                                   "type": "string"
                                               },
                                               "output": {
                                                   "type": "string"
                                               }
                                           },
                                           "required": ["explanation", "output"],
                                           "additionalProperties": False
                                       }
                                   },
                                   "final_output": {
                                       "type": "string"
                                   }
                               },
                               "additionalProperties": False,
                               "required": ["steps", "final_output"]
                           },
                           "strict": True
                       }}

    response = client.chat.completions.create(messages=[{"role": "system",
                                                         "content": dedent(math_tutor_prompt)},
                                                        {"role": "user",
                                                         "content": question}],
                                              model=MODEL, response_format=response_format)

    return response.choices[0].message.content

In [6]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question)
print(result)

{"steps":[{"explanation":"To solve for x, we want to isolate the variable x on one side of the equation. First, we will eliminate the constant on the left side of the equation by subtracting 7 from both sides.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplifying both sides, we have: 8x = -30.","output":"8x = -30"},{"explanation":"Next, we need to isolate x by dividing both sides of the equation by 8.","output":"8x / 8 = -30 / 8"},{"explanation":"Simplifying the equation gives us x = -30/8, which can be simplified further. We simplify -30/8 by dividing both the numerator and the denominator by 2.","output":"x = -15/4 or x = -3.75"}],"final_output":"x = -3.75"}


In [9]:
import json
from IPython.display import Math, display


def print_math_response(response):
    response = json.loads(response)
    steps = response["steps"]
    final_output = response["final_output"]

    for i in range(len(steps)):
        print(f"Steps: {i + 1}: {steps[i]['explanation']}\n")
        display(steps[i]["output"])

    print(f"Final answer:\n\n")
    display(Math(final_output))

In [10]:
print_math_response(result)

Steps: 1: To solve for x, we want to isolate the variable x on one side of the equation. First, we will eliminate the constant on the left side of the equation by subtracting 7 from both sides.



'8x + 7 - 7 = -23 - 7'

Steps: 2: Simplifying both sides, we have: 8x = -30.



'8x = -30'

Steps: 3: Next, we need to isolate x by dividing both sides of the equation by 8.



'8x / 8 = -30 / 8'

Steps: 4: Simplifying the equation gives us x = -30/8, which can be simplified further. We simplify -30/8 by dividing both the numerator and the denominator by 2.



'x = -15/4 or x = -3.75'

Final answer:




<IPython.core.display.Math object>

# Using the SDK parse helper


In [11]:
from pydantic import BaseModel


class MathReasoning(BaseModel):

    class Steps(BaseModel):
        explanation: str
        output: str

    steps: list[Steps]
    final_output: str


def get_math_solution(question: str):

    response = client.beta.chat.completions.parse(messages=[{"role": "system",
                                                             "content": dedent(math_tutor_prompt)},
                                                            {"role": "user",
                                                             "content": question}],
                                                  model=MODEL,
                                                  response_format=MathReasoning)

    return response.choices[0].message

In [12]:
result = get_math_solution(question).parsed

In [14]:
print(result)

steps=[Steps(explanation='We start with the equation 8x + 7 = -23. The goal is to isolate the variable x. To do this, we first need to eliminate the constant term on the left side of the equation.', output='8x + 7 = -23'), Steps(explanation="Subtract 7 from both sides of the equation to get rid of the '+ 7'. This gives us: 8x + 7 - 7 = -23 - 7.", output='8x = -30'), Steps(explanation='Now we simplify the right side of the equation. We have 8x = -30. Next, we need to solve for x by dividing both sides of the equation by 8.', output='x = -30 / 8'), Steps(explanation='Now, we can simplify -30 / 8. Both the numerator and the denominator can be divided by 2, which gives us -15 / 4.', output='x = -15/4'), Steps(explanation='Expressing -15/4 as a decimal, we can also write it as -3.75 if desired. Therefore, the solution to the equation is x = -15/4 or x = -3.75.', output='x = -15/4 or x = -3.75')] final_output='x = -15/4 or x = -3.75'


In [15]:
print(dedent(math_tutor_prompt))

You are tutor and an expert in solving mathematical problems.
You will be given a math problem. Solve the problem step-by-step with reasoning. For each step provide an explanation and output


## Example 2: Entity extraction from user input


In [16]:
from enum import Enum
import openai

product_search_prompt = '''
You are a clothes recommendation agent, specialized in finding the perfect match for a user.
You will be provided with a user input, and additional context such as user gender, age group, and season.
You are equipped with a tool to search clothes in a database that matches the user's profile and preferences.
Based on the user input and context, determine the most likely value of parameters to use to search the database.

Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers    

There are a wide range of colors available, but try to stick to regular color names.   
'''


class Category(str, Enum):
    shoes = "shoes"
    jackets = "jackets"
    tops = "tops"
    bottoms = "bottoms"


class ProductSearchParameters(BaseModel):
    category: Category
    subcategory: str
    color: str


def get_response(user_input, context):
    response = client.chat.completions.create(messages=[{"role": "system",
                                                         "content": dedent(product_search_prompt)},
                                                        {"role": "user",
                                                         "content": f"CONTEXT: {context}\n\nUSER INPUT: {user_input}"}],
                                              model=MODEL,
                                              temperature=0,
                                              tools=[openai.pydantic_function_tool(model=ProductSearchParameters,
                                                                                   name="product_search",
                                                                                   description="Search for a match in product database")])

    return response.choices[0].message.tool_calls


example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes"
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40"
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter"
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30"
    }
]

In [17]:
for example in example_inputs:
    print(get_response(user_input=example["user_input"], context=example["context"]))

[ChatCompletionMessageToolCall(id='call_yPzEFC2H61z09cFM8stbKPy6', function=Function(arguments='{"category": "jackets", "subcategory": "winter coats", "color": "blue"}', name='product_search'), type='function'), ChatCompletionMessageToolCall(id='call_Ukbl2W4B9rpKlO7cmUMcDIpd', function=Function(arguments='{"category": "jackets", "subcategory": "parkas", "color": "blue"}', name='product_search'), type='function')]
[ChatCompletionMessageToolCall(id='call_shnFztIsdaLBQWveP5Yse8xR', function=Function(arguments='{"category": "jackets", "subcategory": "rain jackets", "color": "blue"}', name='product_search'), type='function'), ChatCompletionMessageToolCall(id='call_tjG4Zl6L03xerd0Y7NMQ94MI', function=Function(arguments='{"category": "shoes", "subcategory": "boots", "color": "brown"}', name='product_search'), type='function')]
[ChatCompletionMessageToolCall(id='call_Jpo5cdeki0WhsHA7JUAR4xTd', function=Function(arguments='{"category":"shoes","subcategory":"boots","color":"black"}', name='produ